In [1]:
# Take care Seif to Ezzeldin in Path
import pandas as pd
import numpy as np
import random as rnd
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set()

UsageError: Line magic function `%` not found.


In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits = 10 , shuffle = True , random_state =0)

In [3]:
train = pd.read_csv(r"C:\Users\Seif\Desktop\Titanic Competition\train.csv")
test = pd.read_csv(r"C:\Users\Seif\Desktop\Titanic Competition\test.csv")

In [4]:
train["Title"] = train["Name"].str.extract('([A-Za-z]+)\.',expand=False)
title_mapping = {"Mr": 0 , "Miss": 1 , "Mrs": 2 , "Master": 3 , "Dr" : 3 , "Rev": 3 , "Mlle": 3 , "Col": 3 ,"Major": 3 ,
"Countess": 3, "Ms": 3 , "Lady": 3 , "Jonkheer": 3 , "Don": 3 , "Mme": 3 , "Capt": 3 , "Sir": 3 , "Dona": 3 }
train['Title'] = train['Title'].map(title_mapping)
train.drop('Name',axis =1, inplace =True) 
sex_mapping ={"male": 0 , "female": 1}
train['Sex'] = train['Sex'].map(sex_mapping)
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"), inplace = True)
train.loc[train["Age"] <= 16 , "Age"] = 0
train.loc[(train["Age"]) > 16 & (train["Age"] <= 26)  , "Age"] = 1
train.loc[(train["Age"]) > 26 & (train["Age"] <= 36) , "Age"] = 2
train.loc[(train["Age"]) > 36 & (train["Age"] <= 62) , "Age"] = 3
train.loc[train["Age"] > 62 , "Age"] = 4
train["Embarked"] = train["Embarked"].fillna("S")
train.loc[train["Embarked"] == "S" , "Embarked"] = 0
train.loc[train["Embarked"] == "C" , "Embarked"] = 1
train.loc[train["Embarked"] == "Q",  "Embarked"] = 2
train["Fare"].fillna(train.groupby("Pclass")["Fare"].transform("median"), inplace = True)
train.loc[train["Fare"] <= 17 , "Fare"] = 0
train.loc[(train["Fare"] > 17) & (train["Fare"] <= 30) , "Fare"] = 1
train.loc[(train["Fare"] > 30) & (train["Fare"] <= 100) , "Fare"] = 2
train.loc[train["Fare"] > 100 , "Fare"] = 3
train["Cabin"] = train["Cabin"].str[:1]
cabin_mapping = {"A" : 0 , "B" : 0.4 , "C" : 0.8 , "D" : 1.2 , "E" : 1.6 , "F" : 2 , "G" : 2.4 , "T" : 2.8 }
train["Cabin"] = train["Cabin"].map(cabin_mapping)
train["Cabin"].fillna(train.groupby("Pclass")["Cabin"].transform("median") , inplace = True)
train["FamilySize"] = train["SibSp"] + train["Parch"] + 1
family_mapping = { 1 : 0 , 2 : 0.4 , 3 : 0.8 , 4 : 1.2 , 5 : 1.6 , 6 : 2 , 7 : 2.4 , 8 : 2.8 , 9 : 3.2 , 10 : 3.6 , 11 : 4 }
train["FamilySize"] = train["FamilySize"].map(family_mapping)
train = train.drop( ["Ticket" , "SibSp" , "Parch"] , axis = 1 )
train = train.drop(['PassengerId'] , axis = 1 )
train_data = train.drop('Survived', axis = 1)
target = train['Survived']

In [5]:
test["Title"] = test["Name"].str.extract('([A-Za-z]+)\.',expand=False)
title_mapping = {"Mr": 0 , "Miss": 1 , "Mrs": 2 , "Master": 3 , "Dr" : 3 , "Rev": 3 , "Mlle": 3 , "Col": 3 ,"Major": 3 ,
"Countess": 3, "Ms": 3 , "Lady": 3 , "Jonkheer": 3 , "Don": 3 , "Mme": 3 , "Capt": 3 , "Sir": 3 , "Dona": 3 }
test['Title'] = test['Title'].map(title_mapping)
test.drop('Name',axis =1, inplace =True)       

In [6]:
sex_mapping ={"male": 0 , "female": 1}
test['Sex'] = test['Sex'].map(sex_mapping)

In [7]:
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"), inplace = True)
test.loc[test["Age"] <= 16 , "Age"] = 0
test.loc[(test["Age"]) > 16 & (test["Age"] <= 26)  , "Age"] = 1
test.loc[(test["Age"]) > 26 & (test["Age"] <= 36) , "Age"] = 2
test.loc[(test["Age"]) > 36 & (test["Age"] <= 62) , "Age"] = 3
test.loc[test["Age"] > 62 , "Age"] = 4

In [8]:
test["Embarked"] = test["Embarked"].fillna("S")
test.loc[test["Embarked"] == "S" , "Embarked"] = 0
test.loc[test["Embarked"] == "C" , "Embarked"] = 1
test.loc[test["Embarked"] == "Q",  "Embarked"] = 2

In [9]:
test["Fare"].fillna(test.groupby("Pclass")["Fare"].transform("median"), inplace = True)
test.loc[test["Fare"] <= 17 , "Fare"] = 0
test.loc[(test["Fare"] > 17) & (test["Fare"] <= 30) , "Fare"] = 1
test.loc[(test["Fare"] > 30) & (test["Fare"] <= 100) , "Fare"] = 2
test.loc[test["Fare"] > 100 , "Fare"] = 3

In [10]:
test["Cabin"] = test["Cabin"].str[:1]
cabin_mapping = {"A" : 0 , "B" : 0.4 , "C" : 0.8 , "D" : 1.2 , "E" : 1.6 , "F" : 2 , "G" : 2.4 , "T" : 2.8 }
test["Cabin"] = test["Cabin"].map(cabin_mapping)
test["Cabin"].fillna(test.groupby("Pclass")["Cabin"].transform("median") , inplace = True)

In [11]:
test["FamilySize"] = test["SibSp"] + test["Parch"] + 1
family_mapping = { 1 : 0 , 2 : 0.4 , 3 : 0.8 , 4 : 1.2 , 5 : 1.6 , 6 : 2 , 7 : 2.4 , 8 : 2.8 , 9 : 3.2 , 10 : 3.6 , 11 : 4 }
test["FamilySize"] = test["FamilySize"].map(family_mapping)

In [12]:
test = test.drop( ["Ticket" , "SibSp" , "Parch"] , axis = 1 )

In [13]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [14]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
Title         0
FamilySize    0
dtype: int64

In [15]:
clf = SVC()
scoring = 'accuracy'
score = cross_val_score(clf , train_data , target , cv = k_fold , n_jobs = 1 , scoring = scoring)
print(score)

[0.85555556 0.80898876 0.82022472 0.80898876 0.82022472 0.82022472
 0.84269663 0.84269663 0.83146067 0.86516854]


C:\Users\Seif\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Seif\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Seif\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Seif\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: 

In [16]:
clf = SVC()
clf.fit(train_data , target)
test_data = test.drop("PassengerId" , axis = 1).copy()
prediction = clf.predict(test_data)

C:\Users\Seif\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [17]:
submission = pd.DataFrame({"PassengerId" : test["PassengerId"] , "Survived" : prediction})
submission.to_csv('submission.csv',index = False)

In [18]:
submission = pd.read_csv('submission.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
